<a href="https://colab.research.google.com/github/jhonatanyara/SenalesySistemas/blob/main/Taller2FOURIER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Instalar librerías de Python y herramientas del sistema (FFmpeg)
!apt-get update -qq && apt-get install -y ffmpeg
!pip install -q streamlit numpy scipy matplotlib yt-dlp pydub

# 2. Descargar y configurar Cloudflared (para ver la app desde Colab)
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64
!mv cloudflared-linux-amd64 /usr/local/bin/cloudflared

# 3. Crear carpeta para las páginas
import os
os.makedirs('pages', exist_ok=True)
print("✅ Entorno configurado correctamente.")

In [ ]:
%%writefile 0_Inicio.py
import streamlit as st

# Configuración general
st.set_page_config(
    page_title="Solución Taller SyS",
    page_icon="📘",
    layout="wide"
)

# Estilos CSS y Encabezado
st.markdown("""
    <style>
    .main {background-color: #f5f5f5;}
    .stButton>button {width: 100%;}
    </style>
    <div style='background-color:#003366;padding:15px;border-radius:10px'>
        <h1 style='color:white;text-align:center;'>Solución Taller: Señales y Sistemas</h1>
        <h3 style='color:#FFD700;text-align:center;'>Universidad Nacional de Colombia - Sede Manizales</h3>
        <h4 style='color:white;text-align:center;'>Estudiante: Jhonatan Yara Lopez</h4>
        <h5 style='color:#B0C4DE;text-align:center;'>Profesor: Andrés Marino Álvarez Meza, Ph.D.</h5>
    </div>
""", unsafe_allow_html=True)

st.markdown("""
### 📘 Introducción al Taller

Este dashboard presenta la **solución computacional e interactiva** a los ejercicios propuestos en el taller de la asignatura.

#### 📂 Contenido del Taller:

* **📻 Ejercicio 1: Modulación AM (DSB-SC)**
    * Simulación de transmisión de audio real.
    * Implementación de mezclador y **filtrado ideal mediante FFT**.
    * Recuperación de mensaje.

* **⚙️ Ejercicio 2: Sistemas Dinámicos**
    * Análisis de sistema **Masa-Resorte-Amortiguador**.
    * Analogía con circuitos eléctricos (RLC).
    * Respuesta en frecuencia y transitoria.

---
👈 **Por favor, selecciona un ejercicio en el menú lateral para ver la solución.**
""")

st.sidebar.info("Navegación del Taller")
st.sidebar.markdown("Selecciona una página arriba 👆")

In [ ]:
%%writefile pages/1_📻_Modulacion_AM.py
import streamlit as st
import numpy as np
from scipy.fft import fft, ifft, fftshift, fftfreq
from scipy.io import wavfile
import matplotlib.pyplot as plt
import yt_dlp
import os
import subprocess

st.set_page_config(page_title="Solución: Modulación AM", page_icon="📻", layout="wide")

st.markdown("""
    <h2 style='text-align: center; color: #003366;'>📻 Solución Ejercicio 1: Modulación DSB-SC</h2>
    <p>Se implementa un sistema de Doble Banda Lateral con Portadora Suprimida, usando filtrado ideal en frecuencia para la demodulación.</p>
""", unsafe_allow_html=True)

# --- SIDEBAR ---
st.sidebar.header("🎛️ Configuración de Señal")
url = st.sidebar.text_input("URL YouTube", "https://www.youtube.com/watch?v=5qap5aO4i9A")
start_s = st.sidebar.number_input("Segundo Inicio", value=30)
fc = st.sidebar.slider("Frecuencia Portadora (Hz)", 1000, 15000, 5000)
cutoff = st.sidebar.slider("Frecuencia Corte Filtro (Hz)", 1000, 8000, 4000)

# --- FUNCIONES ---
@st.cache_data
def get_audio(url, start, dur=5):
    """Descarga y procesa audio de YT"""
    out = "audio.wav"
    # Opciones actualizadas para yt-dlp
    opts = {
        'format':'bestaudio/best',
        'outtmpl':'temp',
        'quiet':True,
        'overwrites':True
    }
    try:
        if os.path.exists(out): os.remove(out)
        with yt_dlp.YoutubeDL(opts) as ydl:
            ydl.download([url])

        # Convertir a WAV mono 44.1kHz con ffmpeg
        subprocess.run(
            ['ffmpeg', '-i', 'temp', '-ss', str(start), '-t', str(dur),
             '-ac', '1', '-ar', '44100', out, '-y'],
            stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL
        )

        if not os.path.exists(out): return None, None

        fs, data = wavfile.read(out)
        # Normalizar
        if data.dtype != np.float32:
            data = data.astype(np.float32) / np.max(np.abs(data))
        return fs, data
    except Exception as e:
        st.error(f"Error procesando audio: {e}")
        return None, None

def plot_tf(t, sig, fs, title, color):
    """Grafica Tiempo y Frecuencia"""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 3.5))
    # Tiempo
    ax1.plot(t, sig, color=color, lw=0.8)
    ax1.set_title(f"Tiempo: {title}")
    ax1.set_xlabel("Tiempo (s)")
    ax1.grid(alpha=0.3)
    # Frecuencia
    N = len(sig)
    Y = fftshift(fft(sig))
    f = fftshift(fftfreq(N, 1/fs))
    ax2.plot(f, np.abs(Y)/N, color=color, lw=0.8)
    ax2.set_title(f"Espectro: {title}")
    ax2.set_xlabel("Frecuencia (Hz)")
    ax2.set_xlim(-fs/2, fs/2)
    ax2.grid(alpha=0.3)
    st.pyplot(fig)

# --- APP ---
col1, col2 = st.columns([1, 3])
with col1:
    if st.button("▶️ Ejecutar Solución", type="primary"):
        st.session_state['run'] = True

if st.session_state.get('run', False):
    with st.spinner("Descargando y procesando audio..."):
        fs, m_t = get_audio(url, start_s)

    if m_t is not None:
        N = len(m_t)
        t = np.linspace(0, 5, N)

        st.markdown("### 1. Señal Mensaje Original $m(t)$")
        st.audio(m_t, sample_rate=fs)
        plot_tf(t, m_t, fs, "Mensaje Original", "#1E90FF")

        # Modulación
        c_t = np.cos(2*np.pi*fc*t)
        y_t = m_t * c_t

        st.markdown("### 2. Señal Modulada (DSB-SC) $y(t) = m(t) \cdot \cos(\omega_c t)$")
        st.audio(y_t, sample_rate=fs)
        plot_tf(t, y_t, fs, "Modulada DSB-SC", "#28a745")

        # Demodulación (Mezcla)
        y_mix = y_t * c_t

        # Filtro Ideal FFT
        Y_f = fft(y_mix)
        freqs = fftfreq(N, 1/fs)
        # Filtro ideal: 1 si |f| < cutoff, 0 si no
        mask = np.abs(freqs) < cutoff

        # Aplicar filtro y ganancia (x2 para compensar la amplitud 1/2 del coseno^2)
        m_rec = np.real(ifft(Y_f * mask)) * 2

        st.markdown("### 3. Señal Recuperada (Mezcla + Filtro Ideal)")
        st.audio(m_rec, sample_rate=fs)
        plot_tf(t, m_rec, fs, "Señal Recuperada", "#dc3545")

        st.success("✅ Ejercicio resuelto: El mensaje ha sido recuperado eliminando las réplicas de alta frecuencia.")
    else:
        st.error("No se pudo cargar el audio. Intenta con otro link o verifica ffmpeg.")
else:
    st.info("Configura los parámetros en el menú y presiona 'Ejecutar Solución'.")

In [ ]:
%%writefile pages/2_⚙️_Sistema_Masa_Resorte.py
import streamlit as st
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt

st.set_page_config(page_title="Solución: Sistemas Dinámicos", page_icon="⚙️", layout="wide")

st.markdown("""
    <h2 style='text-align: center; color: #003366;'>⚙️ Solución Ejercicio 2: Sistemas de 2do Orden</h2>
    <p>Análisis de respuesta transitoria y equivalencias electromecánicas (Masa-Resorte ↔ RLC Serie).</p>
""", unsafe_allow_html=True)

# --- SIDEBAR ---
st.sidebar.header("Parámetros del Sistema")
caso = st.sidebar.selectbox("Tipo de Respuesta", ["Subamortiguado (Oscilatorio)", "Crítico", "Sobreamortiguado"])

if "Sub" in caso:
    zeta = st.sidebar.slider("Factor de Amortiguamiento (ζ)", 0.05, 0.95, 0.3)
elif "Crít" in caso:
    zeta = 1.0
    st.sidebar.info("ζ fijado en 1.0 (Crítico)")
else:
    zeta = st.sidebar.slider("Factor de Amortiguamiento (ζ)", 1.05, 5.0, 1.5)

wn = st.sidebar.slider("Frecuencia Natural (ωn) [rad/s]", 1.0, 20.0, 5.0)
lazo = st.sidebar.radio("Configuración de Control", ["Lazo Abierto", "Lazo Cerrado"])

# --- CÁLCULOS ---
# Mecánico (Normalizado m=1 kg)
m = 1.0
k = (wn**2) * m       # Rigidez del resorte
c = 2 * zeta * wn * m # Coeficiente de amortiguamiento

# Eléctrico (Serie RLC, asumiendo C=1mF de referencia)
C_el = 0.001          # 1000 uF
L_el = 1 / (wn**2 * C_el)
R_el = 2 * zeta * wn * L_el

# Funciones de Transferencia
# G(s) = wn^2 / (s^2 + 2*zeta*wn*s + wn^2)
num = [wn**2]
den = [1, 2*zeta*wn, wn**2]

if lazo == "Lazo Cerrado":
    # Feedback unitario: G_cl = G / (1+G)
    den_sys = [den[0], den[1], den[2] + num[0]]
    sys = signal.TransferFunction(num, den_sys)
    title_plot = "Respuesta Lazo Cerrado"
else:
    sys = signal.TransferFunction(num, den)
    title_plot = "Respuesta Lazo Abierto"

# --- VISUALIZACIÓN ---
st.subheader("1. Analogía Electromecánica")
col1, col2, col3 = st.columns(3)

with col1:
    st.markdown("**Sistema Mecánico**")
    st.metric("Masa (m)", "1.0 kg")
    st.metric("Resorte (k)", f"{k:.2f} N/m")
    st.metric("Amortiguador (c)", f"{c:.2f} N·s/m")

with col2:
    st.markdown("**Circuito RLC Serie**")
    st.metric("Inductancia (L)", f"{L_el*1000:.1f} mH")
    st.metric("Capacitancia (C)", f"{C_el*1000:.1f} mF")
    st.metric("Resistencia (R)", f"{R_el:.2f} Ω")

with col3:
    st.markdown("**Parámetros de Control**")
    st.metric("Frecuencia (ωn)", f"{wn} rad/s")
    st.metric("Amortiguamiento (ζ)", f"{zeta:.2f}")

st.markdown("---")
st.subheader("2. Análisis Gráfico")

tab1, tab2, tab3 = st.tabs(["📈 Respuesta al Escalón", "🔊 Diagrama de Bode", "✖️ Polos y Ceros"])

with tab1:
    t, y = signal.step(sys)
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(t, y, lw=2, color='#003366', label=f'ζ={zeta}')
    ax.set_title(f"Respuesta Transitoria - {title_plot}")
    ax.set_xlabel("Tiempo (s)")
    ax.set_ylabel("Amplitud")
    ax.grid(True, alpha=0.5)
    ax.legend()
    st.pyplot(fig)

with tab2:
    w, mag, phase = signal.bode(sys)
    fig, (ax1, ax2) = plt.subplots(2, 1, sharex=True, figsize=(10, 5))
    ax1.semilogx(w, mag, color='purple', lw=2)
    ax1.set_ylabel("Magnitud (dB)")
    ax1.set_title("Diagrama de Bode")
    ax1.grid(True, which="both", alpha=0.3)

    ax2.semilogx(w, phase, color='purple', lw=2)
    ax2.set_ylabel("Fase (deg)")
    ax2.set_xlabel("Frecuencia (rad/s)")
    ax2.grid(True, which="both", alpha=0.3)
    st.pyplot(fig)

with tab3:
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.scatter(np.real(sys.poles), np.imag(sys.poles), marker='x', s=150, color='red', label='Polos')
    if len(sys.zeros) > 0:
        ax.scatter(np.real(sys.zeros), np.imag(sys.zeros), marker='o', s=150, facecolors='none', edgecolors='blue', label='Ceros')
    ax.axhline(0, color='k', lw=1)
    ax.axvline(0, color='k', lw=1)
    ax.grid(True, linestyle='--')
    ax.set_title("Mapa de Polos y Ceros (Plano S)")
    ax.legend()
    st.pyplot(fig)

In [ ]:
# 5. Ejecutar Streamlit en background y abrir túnel
import time
print("⏳ Iniciando Streamlit (Solución Taller)...")

# Ejecutar Streamlit en segundo plano
!streamlit run 0_Inicio.py &>/dev/null &

# Esperar unos segundos a que cargue
time.sleep(5)

# Crear el túnel
print("🚀 Generando link de acceso...")
!cloudflared tunnel --url http://localhost:8501